<a href="https://colab.research.google.com/github/DeepLearningSaeid/Grad/blob/main/Optimized%20SWAG_pythorch_needs_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install autokeras -q





In [49]:
import numpy as np

def x_1(x):
    return np.power(x, 1)

def dx_1(x):
    return np.ones_like(x)

def x_2(x):
    return np.power(x, 2) / 2

def dx_2(x):
    return x

def x_2_(x):
    return np.power(x, 2) / 24

def dx_2_(x):
    return x / 12

def x_2__(x):
    return np.power(x, 2) / 720

def dx_2__(x):
    return x / 360

class OptimizedModel:
    def __init__(self, input_dim, hidden_dim, output_dim):
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim

        self.dense1_x1_w = np.random.randn(input_dim, hidden_dim) * np.sqrt(2. / input_dim)
        self.dense1_x1_b = np.zeros(hidden_dim)
        self.dense1_x2_w = np.random.randn(input_dim, hidden_dim) * np.sqrt(2. / input_dim)
        self.dense1_x2_b = np.zeros(hidden_dim)
        self.dense2_w = np.random.randn(hidden_dim * 2, hidden_dim) * np.sqrt(2. / (hidden_dim * 2))
        self.dense2_b = np.zeros(hidden_dim)
        self.dense3_w = np.random.randn(hidden_dim, hidden_dim) * np.sqrt(2. / hidden_dim)
        self.dense3_b = np.zeros(hidden_dim)
        self.dense_out_w = np.random.randn(hidden_dim * 4, hidden_dim) * np.sqrt(2. / (hidden_dim * 4))
        self.dense_out_b = np.zeros(hidden_dim)
        self.final_out_w = np.random.randn(hidden_dim, output_dim) * np.sqrt(2. / hidden_dim)
        self.final_out_b = np.zeros(output_dim)

    def forward(self, x):
        self.x = x
        self.z1 = np.dot(x, self.dense1_x1_w) + self.dense1_x1_b
        self.x1 = x_1(self.z1)
        self.z2 = np.dot(x, self.dense1_x2_w) + self.dense1_x2_b
        self.x2 = x_2(self.z2)
        self.concat1 = np.concatenate((self.x1, self.x2), axis=1)

        self.z3 = np.dot(self.concat1, self.dense2_w) + self.dense2_b
        self.x3 = x_2_(self.z3)
        self.z4 = np.dot(self.x3, self.dense3_w) + self.dense3_b
        self.x4 = x_2__(self.z4)

        self.concat2 = np.concatenate((self.concat1, self.x3, self.x4), axis=1)
        self.out1 = np.dot(self.concat2, self.dense_out_w) + self.dense_out_b
        self.out = np.dot(self.out1, self.final_out_w) + self.final_out_b

        return self.out

    def backward(self, y_true):
        learning_rate = 0.01

        # Compute gradient of loss w.r.t outputs
        d_loss = 2 * (self.out - y_true)

        # Backprop through final output layer
        d_final_out_w = self.out1.T @ d_loss
        d_final_out_b = np.sum(d_loss, axis=0)

        # Update final layer weights
        self.final_out_w -= learning_rate * d_final_out_w
        self.final_out_b -= learning_rate * d_final_out_b

        # Propagate through final layer
        d_out1 = d_loss @ self.final_out_w.T

        # Backprop through dense_out layer
        d_dense_out_w = self.concat2.T @ d_out1
        d_dense_out_b = np.sum(d_out1, axis=0)

        # Update dense_out layer weights
        self.dense_out_w -= learning_rate * d_dense_out_w
        self.dense_out_b -= learning_rate * d_dense_out_b

        # Propagate through dense_out
        d_concat2 = d_out1 @ self.dense_out_w.T

        # Separate gradients for earlier layers from concat2
        d_concat1 = d_concat2[:, :self.hidden_dim*2]
        d_x3 = d_concat2[:, self.hidden_dim*2:self.hidden_dim*3] * dx_2_(self.z3)
        d_x4 = d_concat2[:, self.hidden_dim*3:] * dx_2__(self.z4)

        # Propagate through dense3
        d_dense3_w = self.x3.T @ d_x4
        d_dense3_b = np.sum(d_x4, axis=0)
        d_x3 += d_x4 @ self.dense3_w.T

        # Update dense3 layer weights
        self.dense3_w -= learning_rate * d_dense3_w
        self.dense3_b -= learning_rate * d_dense3_b

        # Propagate through dense2
        d_dense2_w = self.concat1.T @ d_x3
        d_dense2_b = np.sum(d_x3, axis=0)

        # Update dense2 layer weights
        self.dense2_w -= learning_rate * d_dense2_w
        self.dense2_b -= learning_rate * d_dense2_b

        # Separate gradients for x1 and x2 from concat1
        d_x1 = d_concat1[:, :self.hidden_dim] * dx_1(self.z1)
        d_x2 = d_concat1[:, self.hidden_dim:] * dx_2(self.z2)

        # Propagate through dense1_x1
        d_dense1_x1_w = self.x.T @ d_x1
        d_dense1_x1_b = np.sum(d_x1, axis=0)

        # Update dense1_x1 layer weights
        self.dense1_x1_w -= learning_rate * d_dense1_x1_w
        self.dense1_x1_b -= learning_rate * d_dense1_x1_b

        # Propagate through dense1_x2
        d_dense1_x2_w = self.x.T @ d_x2
        d_dense1_x2_b = np.sum(d_x2, axis=0)

        # Update dense1_x2 layer weights
        self.dense1_x2_w -= learning_rate * d_dense1_x2_w
        self.dense1_x2_b -= learning_rate * d_dense1_x2_b

